In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip install transformers
!pip install datasets

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
from transformers import TextDataset, DataCollatorForLanguageModeling
from transformers import GPT2LMHeadModel
from transformers import Trainer, TrainingArguments
from transformers import PreTrainedTokenizerFast

def load_dataset(file_path, tokenizer, block_size = 128):
    dataset = TextDataset(
        tokenizer = tokenizer,
        file_path = file_path,
        block_size = block_size,
    )
    return dataset

def load_data_collator(tokenizer, mlm = False):
    data_collator = DataCollatorForLanguageModeling(
        tokenizer = tokenizer,
        mlm = mlm,
    )
    return data_collator

def train(train_file_path, model_name, output_dir, overwrite_output_dir,
          per_device_train_batch_size, num_train_epochs, save_steps):
    tokenizer = PreTrainedTokenizerFast.from_pretrained(model_name,
                bos_token = '</s>', eos_token = '</s>', unk_token = '<unk>',
                pad_token = '<pad>', mask_token = '<mask>')
    train_dataset = load_dataset(train_file_path, tokenizer)
    data_collator = load_data_collator(tokenizer)

    tokenizer.save_pretrained(output_dir, legacy_format = False)
    model = GPT2LMHeadModel.from_pretrained(model_name)
    model.save_pretrained(output_dir)

    training_args = TrainingArguments(
        output_dir = output_dir,
        overwrite_output_dir = overwrite_output_dir,
        per_device_eval_batch_size = per_device_train_batch_size,
        num_train_epochs = num_train_epochs,
    )

    trainer = Trainer(
        model = model,
        args = training_args,
        data_collator = data_collator,
        train_dataset = train_dataset,
    )

    trainer.train()
    trainer.save_model()

train_file_path = '/content/drive/MyDrive/Colab Notebooks/팀프로젝트/빅데이터 지능형 서비스과정(최종프로젝트)/KoGPT2_FineTunning/문학광장(수필)/Data/Crawling(문학광장_수필).txt'
model_name = 'skt/kogpt2-base-v2'
output_dir = '/content/drive/MyDrive/Colab Notebooks/팀프로젝트/빅데이터 지능형 서비스과정(최종프로젝트)/KoGPT2_FineTunning/문학광장(수필)/모델'
overwrite_output_dir = False
per_device_train_batch_size = 8
num_train_epochs = 5.0
save_steps = 500

train(train_file_path = train_file_path,
      model_name = model_name,
      output_dir = output_dir,
      overwrite_output_dir = overwrite_output_dir,
      per_device_train_batch_size = per_device_train_batch_size,
      num_train_epochs = num_train_epochs,
      save_steps = save_steps
)

In [ ]:
from transformers import PreTrainedTokenizerFast, GPT2LMHeadModel
from tqdm.notebook import tqdm

def load_model(model_path):
  model = GPT2LMHeadModel.from_pretrained(model_path)
  return model

def load_tokenizer(tokenizer_path):
  tokenizer = PreTrainedTokenizerFast.from_pretrained(tokenizer_path)
  return tokenizer

def generate_text(sequence, max_lenth):
  model_path = '/content/drive/MyDrive/Colab Notebooks/팀프로젝트/빅데이터 지능형 서비스과정(최종프로젝트)/KoGPT2_FineTunning/문학광장(수필)/모델/checkpoint-12500'
  model = load_model(model_path)
  tokenizer = load_tokenizer(model_path)
  ids = tokenizer.encode(f'{sequence},', return_tensors = 'pt')
  final_outputs = model.generate(
      ids,
      do_sample = True,
      max_length = max_length,
      pad_token_id = model.config.pad_token_id,
      top_k = 5,
      top_p = 0.90,
      repetition_penalty = 2.0,
  )
  return tokenizer.decode(final_outputs[0])
  # return tokenizer.decode(final_outputs[0], skip_special_tokens = True)

sequence = '코끼리 앞에서 남자가 사진을 찍고있다.'
max_length = 64
sentence_list = []
# print('input : ' + sequence + ' ' + refer)
# for i in tqdm(range(5)):
sentence_list.append(generate_text(sequence, max_length))
sentence = generate_text(sequence, max_length)

In [ ]:
sentence = sentence.replace('\n', ' ')
sentence = sentence.replace('.', '. ')
sentence = sentence.replace('"', '')
sentence = sentence.replace('<unk>', '')
sentence = sentence.replace('?', '? ')
sentence = sentence.replace('!', '! ')
sentence = sentence.replace('  ', ' ')
sentence = sentence.replace('  ', ' ')
sentence = sentence.replace('  ', ' ')
sentence = sentence.replace('다. ', '다.\n')
print(sentence)

코끼리 앞에서 남자가 사진을 찍고있다.
, 그는 아무 말도 없이 고개를 갸웃거렸다.
남자는 입을 딱 벌리고는 뒷걸음질 치다가 이내 다시 눈을 질끔 감았다.
순간 나는 깜짝 놀랐던 것이었다.
어이쿠. 나도 모르는 사이에 그 여자가 사라져 버렸네. 뭐라고 대꾸할 수도 없는 노릇이었다.
그러나 지금 생각해도 참 우습지만 그때의 일은 내 나름대로 충격적이라 생각이 든 뒤였다.
정말 아무것도 모르고 그렇게 평범하게 살던 내가 몇 년이 지나도록 기억조차 나지 않는다니 난감하고 황당했다.
그런데 이상하게도 아직도 생생하다.
중학교 시절이나 대학 시절에나 그리고 지금도 이렇게 살아있는 것 같다.
나에게는 세상이 어떤 식으로 펼쳐졌는지 전혀 알 수 없지만 분명 분명히 누군가를 탓하기에는 많은 시간이 필요할 것이다.
하지만 그것은 세상을 바라보는 방식이 아니다.
나의 마음가짐은 완전히 변해버린 셈이다.
조금 더 나은 삶을 꿈꾸기 위해서 혹은 자신이 옳기에 따라오지 않고 남이 바라는 것을 하기 위해 사는 삶일 뿐인데 과연 그들이 원하는 대로 살고 있는지 모르겠다라는 결론은 나오지 않을지도 모른다이런 생각을 해보긴 어렵지 않겠어요 그렇다면 왜 하셨으면 하는 바램이기도 하겠네요. 그렇겠죠 정말로 안타깝습니다.
이제 좀 전 당신이 하고 싶은 말하고자 했던 말이야 물론 이 글에서 말씀드릴 게 있다면 다른 분들이 있는 곳에 계좌와 같이 되물장창입니다.
얼핏 들어보지 않은 사람이라면 오십니까 글틴 것은 잘 생각해보면 제게 맞습니다 오래 머뭇거리는 말에 따르면 우리 모두 알고싶었소곱 하면 됩니다.
제가 가진 못아프니 고맙다고 다들 네팔님이 주말씀 주세요 사람에 한 번데기님께 가르침을 받고 싶으니 듣노라면 좋단 오라버렸던 터벅 거림 나는 요즘 세상은 까르시 참고하면 세상 구마음이라고 하지 않앗 오늘부터 시작해야 하나 알려주님의 은사촌께서 너댓말에 틀리지 말아쓰리라 하시기를 듣고 보니까 마흔한 이가 들려주는 건드리옵므라 곱셈 십이오넌 누난 꼬맹댕그립다는 뜻만 아니 읽으렴 백담하라면 안 불러도 부

In [ ]:
ch_sentence = sentence.split(sequence + ', ')[1:]
print(f'입력 값 : {sequence}')
ch_sentence = ch_sentence[0].replace('\n', ' ')
ch_sentence = ch_sentence.replace('.', '. ')
ch_sentence = ch_sentence.replace('"', '')
ch_sentence = ch_sentence.replace('<unk>', '')
ch_sentence = ch_sentence.replace('?', '? ')
ch_sentence = ch_sentence.replace('!', '! ')
ch_sentence = ch_sentence.replace('  ', ' ')
ch_sentence = ch_sentence.replace('  ', ' ')
ch_sentence = ch_sentence.replace('  ', ' ')
ch_sentence = ch_sentence.replace('다. ', '다.\n')
print(ch_sentence)

입력 값 : 바다
천문산 등반로를 찾아보며 저번 주 금요일에 불국사를 다녀온 적이 있었다.
그런데 그 날은 수학여행에서 놓쳤던 일정이 있어서 이번에는 뭘 해야 할지 몰랐었다.
결국 고민에 빠지면서 울컥했던 마음은 잠시 진정되어 여행의 첫날을 보낸 후 버스를 타고 불국사역으로 향했다.
대전에서는 제일 끝에 있는 경주에는 가족들과 친구들이 모두 모여 삼삼오 짝을 이룬 채 사진을 찍고 있었는데 우리 집은 사촌동생과 함께 경주에 가기로 돼 있었지만 나는 차비가 없어서 일단 부모님 차에 태워 두어야 했다.
그래서 친가 친척들에게 부탁해서 우린 자전거를 이용해 왔다.
그러나 우리는 예상외로 간단하고 재미있게 갔지만 역시나 시간이 지날수록 심장이 마구 뛰기 시작하였고 난 다시 가슴이 철렁 내려앉을 듯 아팠으며 친구들과 한참이나 시끄럽게 떠들어대고 말았다.
그러던 중 우연히 친구를 만나서 친구가 다가가자 갑자기 내 마음이 이렇게 말하고 싶어진 것은 오랜만에 본 인터넷 카페를 통해서였다.
글을 읽어보았었는데 어느 블로그 글 게시판에 썼던 글에 들어가 보니까 제목까지 생각이 났는지 검색어 순위가 온통 초록색으로 나와 있었고 그냥 보고 싶어 들어갔었다 라는 댓글도 올라가 있을 수 있었더라. 좋아요. 정말 웃음이 가득찼습니다의 느낌이랄수도있었고 여러 의미로 읽었던 것 같구요 사진 한번 올려봅니다.
그리고 답이에 고마지막으로 보자 뭐 이런 느낌이라며 이걸 인증샷 너무 기대감도 즐거웠죠 제가 써봤으니 올 거 참 재밌음ㅠㅠ. 완전 추천해 사실 지금까지 내가 생각한 것이 궁금했습니다.
지금 생각보다 더 올리고싶은 좀 있네 감사합심장우리가 나오니 감명아침ᄏᄒ소연재작 올리진정말그거지 대박백니다 글을 남겨봐 사랑합니다.
사실 이번 여름에 갔던 게임이야 새로 쓴것 같아ᅲ당첨이였다나올리는 여쭈욱 봐세상영입니까 안열심히 했다며 감동할께 감사의절대 올라 와라 기쁘다 언젠 반톡쏘면 사진. 그래 올릴 수가 많아짐 안녕하달겠네. 소고기감자 먹왕양수 없좋여김천기행2104차비틀려찬히 할필한 8월3 잘 마무리 